## Configuration

In [9]:
from collections import Counter

import pandas as pd

FANMOD_path = "C:\\Users\\User\\source\\repos\\fanmod-cmd\\out\\build\\x64-release"
FANMOD_exe = "LocalFANMOD.exe"

raw_data_folder = './../../data'
raw_data_folder_dataset = '/melanoma_distance_100_XX'
output_dir = './../../fanmod_output'
cache_dir = './../../parse_cache'
results_dir = './../../results/melanoma_distance_100_XX'
motif_size = 3
iterations = 1000
force_run_fanmod = False
force_parse = False

In [10]:
cells_type = {
    0: 'B cell',
    1: 'CD4 APC',
    2: 'CD4 T cell',
    3: 'CD4 Treg',
    4: 'CD8 T cell',
    5: 'DCs',
    6: 'Germinal Center B cell',
    7: 'Hevs',
    8: 'MAC',
    9: 'Memory CD4 T Cell',
    10: 'NK cell',
    11: 'Neutrophil',
    12: 'Stroma',
    13: 'Tumor',
    14: 'Unidentified',
    15: 'Vessels'
}

Distant_metastases_class = {
    'NP': 'NP',
    'NN': 'NN',
    'PN': 'PN',
    'PP': 'PP',
}

class_to_color = {
    'NP': 'green',
    'NN': 'yellow',
    'PN': 'magenta',
    'PP': 'blue',
}

target_classes = ['NP', 'PN']
target_classes_str = 'NP_PN'

## Steps
1. Initiate CISM object
2. Load datasets
3. Initiate CISMTask

In [11]:
from cism.cism import CISM

In [12]:
cism = CISM(fanmod_exe=FANMOD_exe,
            fanmod_path=FANMOD_path,
            network_dataset_root_path=raw_data_folder,
            fanmod_output_root_path=output_dir,
            fanmod_cache_root_path=cache_dir,
            motif_size=motif_size,
            iterations=iterations)

# adding Melanoma dataset
cism.add_dataset(raw_data_folder_dataset, 'Disease', 'Melanoma', force_run_fanmod=force_run_fanmod, force_parse=force_parse, n_jobs=8, quantile_threshold=1)

  0%|          | 0/178 [00:00<?, ?it/s]

  0%|          | 0/178 [00:00<?, ?it/s]

In [13]:
from cism.cism import TissueStateDiscriminativeMotifs
from cism.cism import DiscriminativeFeatureKey

discriminator = TissueStateDiscriminativeMotifs(cism=cism,
                                                tissue_state_csv_path=raw_data_folder + raw_data_folder_dataset + '/patient_class.csv',
                                                tissue_state_to_string=Distant_metastases_class,
                                                common_cells_type=cells_type)

Here we want to evaluate to compare the following:
1. NP versus PN
2. NP versus PP
3. NN versus PN
4. NN versus PP

In [6]:
discriminator.get_patients_class(classes=None).groupby('patient_class').count()

,patient_class_id
patient_class,
NN,20
NP,18
PN,13
PP,17


In [7]:
discover_result = discriminator.discover(extract_by=DiscriminativeFeatureKey.STRUCTURE_AND_CELL_IDENTITIES, classes=target_classes)

In [ ]:
discover_result.plot_number_of_motifs_versus_shared_percentage(class_to_color=class_to_color)

In [8]:
discover_result.discriminative_motifs.drop(['Patient', 'FOV', 'Freq', 'Count', 'Mean_Freq', 'STD', 'z_score', 'p_value', 'Patient_uId'], axis=1).groupby('ID', observed=True).first().to_csv(results_dir +f'/{target_classes_str}_{motif_size}_number_of_motifs_versus_shared_percentage.csv')

In [14]:
results_table = None

In [15]:
from cism.cism import HardDiscriminativeFC, SoftDiscriminativeFC, DiscriminativeFeatureKey
import itertools

for left_class, right_class in itertools.combinations(['NP', 'PN'], 2):
    for th in [0.2, 0.3, 0.4, 0.42, 0.44, 0.46, 0.48, 0.5, 0.6, 0.7, 0.72, 0.74, 0.76, 0.8]:
        try:
            print(f'task: {left_class} - {right_class}')
            featureConf = HardDiscriminativeFC(
                extract_by=DiscriminativeFeatureKey.STRUCTURE_AND_CELL_IDENTITIES,
                use_cells_type_composition=False,
                use_motifs=True,
                shared_percentage=th,
                max_class_features=30,
                labels=[left_class, right_class])

            motif_space_features = (discriminator
                                    .get_features(feature_conf=featureConf,
                                                  exclude_patients=[],
                                                  n_jobs=1))

            optimal_result = (discriminator
                              .analyze_motifs(feature_conf=featureConf,
                                              exclude_patients=[],
                                              n_jobs=1))

            print(f'classes: {[left_class, right_class]} '
                  f'th:{th} score: {optimal_result.get_roc_auc_score()}')

            all_motif_features = []
            for idx, row in motif_space_features.iterrows():
                all_motif_features += row['features']
            all_motif_features = list(set(all_motif_features))

            motif_freq = []
            motif_count = []
            motifs_dataset = discriminator.cism.motifs_dataset
            for motif_id in all_motif_features:
                motif_mean_freq = motifs_dataset[motifs_dataset['ID'] == motif_id]['Freq'].mean()
                motif_mean_count = motifs_dataset[motifs_dataset['ID'] == motif_id]['Count'].mean()
                motif_freq.append(motif_mean_freq)
                motif_count.append(motif_mean_count)

            results_table = pd.concat([pd.DataFrame(
                                            {'th': th,
                                             'task': left_class + '-' + right_class,
                                             'left_class': left_class,
                                             'right_class': right_class,
                                             'roc_auc_score': optimal_result.get_roc_auc_score(),
                                             'all_motif_features': all_motif_features,
                                             'motif_mean_freq': motif_freq,
                                             'motif_mean_count': motif_count}),
                                             results_table], ignore_index=True)
        except:
            break

task: NP - PN


  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

classes: ['NP', 'PN'] th:0.2 score: 0.6047008547008548
task: NP - PN


  0%|          | 0/31 [00:00<?, ?it/s]

...

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

classes: ['NP', 'PN'] th:0.48 score: 0.5213675213675213
task: NP - PN


  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

In [17]:
results_table.to_csv(results_dir+f'/benchmark_across_th_NP_PN_{motif_size}_max_motifs.csv')